import the libraries

In [14]:
import math
import re
from collections import defaultdict, Counter
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import pandas as pd
import glob as gb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

read the training corpora

In [0]:
#function to read the corpus
def readCorpus(directory="Data1/train/*.train", type=0):
  files = []
  author = []
  
  for f in gb.glob(directory):
    with open(f, 'r',encoding="utf8", errors='replace') as doc:
      data = doc.read()
      files = np.append(files, data)
      if type==0:
        if (f[12:][:-9]) == 'dickens':
          author.append("dickens")
        elif (f[12:][:-9]) == 'doyle': 
          author.append("doyle")
        elif (f[12:][:-9]) == 'twain': 
          author.append("twain")
      elif type==1:
        author.append(f[11:][:-5])
          
  return files, author
  #Note that in the corpus, we removed \n and : as they are not phonemes

In [0]:
#read the training file with the author names
files, author=readCorpus(directory="Data1/train/*.train",type=0)
#all the training files were read at once into a list of lists, ditto with authors

In [0]:
#read the test file with their names
testfiles, testname=readCorpus(directory="Data1/test/*.test",type=1)

In [0]:
#this function preprocess any string sent to it removing punctuations
def preprocessCorpus(strng):
  patt=re.compile('[^a-z\s+]',re.MULTILINE)
  strx=patt.sub('',strng.lower())
  strx=' '.join(strx.split())
  return strx

In [0]:
#this function tozenize the corpus into sentences
def tokenizeRawDate(files):
  tokenizedFile = []
  for file in files:
    filex=sent_tokenize(file)
    tokenizedFile.append(filex)
  return tokenizedFile

In [0]:
#this function takes the tokenized corpus and does necessary pre-processing by invoking necessary methods returning a list of lists 
def getPreprocessedCorpus(tokenizedFile):
  #here we have the preprocessed file which is a list for each author
  preprocessedFile=[]
  for file in tokenizedFile:
    preprocessedFile.append([preprocessCorpus(strng) for strng in file])
  return preprocessedFile

In [0]:
#training corpus preprocessing 
tokenizedFile=tokenizeRawDate(files)
preprocessedFile=getPreprocessedCorpus(tokenizedFile)

In [0]:
#test corpus preprocessing 
tokenizedTestFile=tokenizeRawDate(testfiles)
preprocessedTestFile=getPreprocessedCorpus(tokenizedTestFile)

word-gram method

In [0]:
def word_ngrams(sent, n):
    """Givne a sent as str return n-grams as a list of tuple"""
    
    # EXAMPLES 
    # > word_ngrams('hello world', 1)
    # [('hello',), ('world',)]
    # > word_ngrams('hello world', 2)
    # [('<s>', 'hello'), ('hello', 'world'), ('world', '</s>')]

    # YOUR CODE HERE
    tp=()
    listtp= []
    token = tokenize(sent)
    #if it is not 1-gram append <s> to the begining and </s> at the end
    if(n!=1):
        for x in range(n-1):
            token.insert(0,"<s>")
        token.append("</s>")
    
    #generate n-grams using the zip function in python
    ngrams = zip(*[token[i:] for i in range(n)])
    
    #loop through the n-grams generated and create list of tuples
    for ngram in ngrams:
        listtp.append(ngram)
        
    return listtp

In [0]:
def tokenize(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower())

In [0]:
#We declare this method to generate the n-grams assisting the 
def getGram(lines, n):
    bigramst=[]
    for sentences in lines:
      bg=word_ngrams(sentences, n)
      bigramst+=bg
    return bigramst

Question 2.1b collect unigram and bigrams

In [0]:
#get the unigram and bigram  lists for the training corpora, thus we have a list of lists
#for unigrams
unigamsList=[]
for file in preprocessedFile:
  unigamsList.append(getGram(file, 1))
  
#for bigram
bigramList=[]
for file in preprocessedFile:
  bigramList.append(getGram(file, 2))  

In [0]:
#get the unigram and bigram count lists lists for the training corpora
unigramsCountList = []
for unig in unigamsList:
  unigramsCountList.append(Counter(unig))
#for bigrams
bigramsCountList = []
for unig in bigramList:
  bigramsCountList.append(Counter(unig))

Question 2.1c Frequently occuring Unigrams

In [0]:
#frequently occuring unigram
top15UnigList=[]
for unig in unigramsCountList:
  top15UnigList.append(unig.most_common(15))
  #frequently occuring unigram
top15BigList=[]
for bigrm in bigramsCountList:
  top15BigList.append(bigrm.most_common(15))

In [0]:
#this method prints the top 15 unigram for the 3 training corpora
def printUnigramTop15():
  for i in range(0, len(author)):
    print(author[i].upper())
    print("word\tFrequency\tRelative Frequency")
    for x  in top15UnigList[i]:
      print(x[0][0]+"\t"+str(x[1])+"\t\t"+str(round(x[1]/len(unigamsList[i]),6)))
    print("\n")
    

In [39]:
#print the top 15 unigram and their relative frequency to 6 decimal places
printUnigramTop15()

DICKENS
word	Frequency	Relative Frequency
the	65399		0.049075
and	50437		0.037848
to	36450		0.027352
of	36261		0.02721
a	28648		0.021497
i	25114		0.018846
in	24094		0.01808
that	18048		0.013543
was	17259		0.012951
he	16281		0.012217
it	16085		0.01207
his	15922		0.011948
her	12923		0.009697
with	12581		0.009441
you	11694		0.008775


DOYLE
word	Frequency	Relative Frequency
the	71907		0.055901
of	34815		0.027065
and	34240		0.026618
a	30908		0.024028
i	30752		0.023907
to	30247		0.023514
that	21371		0.016614
in	21125		0.016423
it	20650		0.016053
was	18714		0.014548
he	17920		0.013931
you	16880		0.013123
his	14649		0.011388
is	12652		0.009836
had	11259		0.008753


TWAIN
word	Frequency	Relative Frequency
the	67083		0.05156
and	59417		0.045668
a	34064		0.026182
to	32986		0.025353
of	30411		0.023374
i	24181		0.018586
it	23535		0.018089
was	20741		0.015942
in	19537		0.015016
he	18164		0.013961
that	16774		0.012893
you	11961		0.009193
his	10666		0.008198
but	10319		0.007931
for	10071		0.007741




In [0]:
#this method prints the top 15 unigram for the 3 training corpora
def printBigramTop15():
  for i in range(0, len(author)):
    print(author[i].upper())
    print("word\t\tFrequency\tRelative Frequency")
    for x  in top15BigList[i]:
      print(str(x[0])+"\t"+str(x[1])+"\t\t"+str(round(x[1]/len(bigramList[i]),6)))
    print("\n")

In [41]:
printBigramTop15()

DICKENS
word		Frequency	Relative Frequency
('<s>', 'i')	7021		0.005015
('of', 'the')	6553		0.004681
('in', 'the')	6133		0.004381
('<s>', 'the')	4212		0.003009
('to', 'the')	3495		0.002496
('<s>', 'he')	3084		0.002203
('to', 'be')	2978		0.002127
('it', 'was')	2613		0.001866
('on', 'the')	2548		0.00182
('and', 'the')	2440		0.001743
('in', 'a')	2252		0.001609
('<s>', 'but')	2194		0.001567
('with', 'a')	2136		0.001526
('<s>', 'it')	2111		0.001508
('of', 'his')	2071		0.001479


DOYLE
word		Frequency	Relative Frequency
('<s>', 'i')	10223		0.00745
('of', 'the')	8862		0.006458
('in', 'the')	5995		0.004369
('<s>', 'the')	5181		0.003776
('<s>', 'it')	4833		0.003522
('<s>', 'he')	4748		0.00346
('it', 'was')	4053		0.002954
('to', 'the')	3589		0.002616
('it', 'is')	3283		0.002393
('<s>', 'you')	2893		0.002108
('i', 'have')	2760		0.002011
('at', 'the')	2710		0.001975
('it', '</s>')	2600		0.001895
('<s>', 'but')	2481		0.001808
('that', 'i')	2317		0.001689


TWAIN
word		Frequency	Relative Frequency
('

Question 2.1d For each author, build a unigram LM and a bigram LM. In this case we used Lidstone Smoothing with alpha=0.5

In [0]:
class ngram_LM:
    """A class to represent a language model."""

    def __init__(self, n, ngram_counts, vocab, filename=None, unk=False):
        """"Make a n-gram language model, given a vocab and
            data structure for n-gram counts."""
        
        self.n = n 
        
        self.vocab = vocab
        
        self.V = len(vocab)
        
        self.uniV = len(one_gram)
        
        self.ngram_counts = ngram_counts
        
        self.histCount = Counter()
        self.centreWordCount = Counter()
        
        self.filename=filename
        self.p=list(dict(self.ngram_counts).keys())
        
        for key, value in ngram_counts.items():
            self.histCount[key[:-1]] += value
            
        for key, value in ngram_counts.items():
            self.centreWordCount[key[:-1]] += 1

        # YOUR CODE HERE
        # START BY MAKING THE RIGHT COUNTS FOR THIS PARTICULAR self.n     
    #discount method starts here
    def estimate_smoothed_probDs(self, history, word, d = 0.5):
        """Estimate probability of a word given a history with Lidstone smoothing."""
        
        if self.n == 1:
            piUnigram = (d/self.uniV)*(self.V)
            if (self.ngram_counts[word]==0):
              prob=1/(self.V)
            else:
              prob = (max(self.ngram_counts[word]-d,0)/self.uniV) + piUnigram*(1/self.V)
            return prob
        else:
            if type(history) == str:
                history = (history,)
            piUnigram = (d/self.uniV)*(self.V )   
            if (ugCount[word]==0):
              proWord=1/(self.V)
            else:
              proWord = (max(ugCount[word]-d,0)/self.uniV) + piUnigram*(1/(self.V))
            
            
            countOfwordTypes = self.centreWordCount[history]
            piBigram=0
            
            if (self.histCount[history]==0):
              piBigram=0
            else:
              piBigram=(d/self.histCount[history])*countOfwordTypes
            prob=0
            if(self.histCount[history]==0):
              prob=proWord;
              
            else:
              prob = (max(self.ngram_counts[history + (word,)]-d,0)/self.histCount[history]) + piBigram*proWord
            #print (history, word, prob)
            return prob
    
        
    def estimate_smoothed_prob(self, history, word, alpha = 0.5):
        """Estimate probability of a word given a history with Lidstone smoothing."""
        
        if self.n == 1:
            return (alpha + self.ngram_counts[word])/(alpha*self.V + self.uniV) #sum(self.ngram_counts.values()))
            
        else:
            if type(history) == str:
                history = (history,)
            return (alpha + self.ngram_counts[history + (word,)])/(alpha*self.V + self.histCount[history])
          

    def logP(self, history, word, alpha = 0.5):
        """Return base-2 log probablity."""
        
        prob = self.estimate_smoothed_prob(history, word, alpha)
        
        if prob == 0:
            return 0
        
        return math.log(prob, 2)
      
    def logPDiscount(self, history, word, d = 0.5):
        """Return base-2 log probablity."""
        
        prob = self.estimate_smoothed_probDs(history, word, d)
        
        if prob == 0:
            return 0
        
        return math.log(prob, 2)
    
    
    
    def logP_unsmoothed(self, history, word):
        """Return base-2 log probablity."""
        prob = self.estimate_smoothed_prob(history, word)
        
        if prob == 0:
            return 0
        
        return math.log(prob, 2)
    


    def score_sentence(self, sentence):
        """Given a sentence, return score."""
        prob = 0;
        
        for gram in sentence:
            prob += -self.logP(gram[0], gram[1])
            
        return prob/len(sentence)


 
    def test_LM(self):
        """Test whether or not the probability mass sums up to one."""
        
        precision = 10**-8
                 
        if self.n == 1:
                 
            P_sum = sum(self.estimate_prob('', w) for w in self.vocab)
            
            assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one.'
                 
        elif self.n == 2:
            histories = ['the', 'in', 'at', 'blue', 'white']
                 
            for h in histories:

                P_sum = sum(self.estimate_prob(h, w) for w in self.vocab)
                
                assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one for history' + h
                     
        print('TEST SUCCESSFUL!')
        



    def test_smoohted_LM(self):
        """Test whether or not the smoothed probability mass sums up to one."""
        precision = 10**-8
        
        if self.n == 1:
                 
            P_sum = sum(self.estimate_smoothed_prob('', w) for w in self.vocab)
            
            assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one.'
                 
        elif self.n == 2:
            histories = ['the', 'in', 'at', 'blue', 'white']
            
            for h in histories:
                P_sum = sum(self.estimate_smoothed_prob(h, w) for w in self.vocab)
                
                assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one for history "' + h + '"'
        
        print('TEST SUCCESSFUL!')
        
       
    def test_Discount(self):
        """Test whether or not the smoothed probability mass sums up to one."""
        precision = 10**-8
        
        if self.n == 1:
                 
            P_sum = sum(self.estimate_smoothed_probDs('', w) for w in self.vocab)
            
            assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one.'
                 
        elif self.n == 2:
            histories = ['the', 'in', 'at', 'blue', 'white']
            
            for h in histories:
                P_sum = sum(self.estimate_smoothed_probDs(h, w) for w in self.vocab)
                assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one for history "' + h + '"'
        
        print('TEST SUCCESSFUL!')
        
            
            
    def perplexity_smoothed(self, corpus, alpha):
        M = len(corpus)        
        
        if self.n == 1:
            likelihood = sum(self.logP('', x,alpha) for x in corpus)
        else:
            likelihood = sum(self.logP(x[:-1], x[-1],alpha) for x in corpus)
            
        perplexity = pow(2, (-likelihood/M))
        return perplexity
    
    
    def perplexity_unsmoothed(self, corpus):
        M = len(corpus)        
        
        if self.n == 1:
            likelihood = sum(self.logP_unsmoothed('', x) for x in corpus)
        else:
            likelihood = sum(self.logP_unsmoothed(x[:-1], x[-1]) for x in corpus)
            
        perplexity = pow(2, (-likelihood/M))
        return perplexity
      
    def perplexity_discount(self, corpus, d=0.5):
        M = len(corpus)        
        
        if self.n == 1:
            likelihood = sum(self.logPDiscount('', x,d) for x in corpus)
        else:
            likelihood = sum(self.logPDiscount(x[:-1], x[-1],d) for x in corpus)
            
        perplexity = pow(2, (-likelihood/M))
        return perplexity

Question 2.1 e

In [0]:
#get the unigram and bigram  lists for the test corpora
unigamsTestList=[]
for file in preprocessedTestFile:
  unigamsTestList.append(getGram(file, 1))
  
#for bigram
bigramTestList=[]
for file in preprocessedTestFile:
  bigramTestList.append(getGram(file, 2))  

In [0]:
#get the unigram vocab for the corpus
unigramVocabList=[]
bigramVocabList=[]
cnt=0;
for x in author:
  unigramVocabList.append(list(unigramsCountList[cnt].keys()))
  
  biVocab = []
  for key in unigramsCountList[cnt].keys():
    biVocab.extend(tokenize(str(key)))
  biVocab.append('<s>')
  biVocab.append('</s>')
  bigramVocabList.append(biVocab)
  cnt=cnt+1
  
  

In [45]:
print(len(unigramVocabList[0]), author[0])

30175 dickens


In [0]:
# ONCE YOU HAVE N-GRAN COUNTS AND VOCAB, 
# YOU CAN BUILD LM OBJECTS AS ... 
one_gram = unigamsList[0]
ugCount=unigramsCountList[0]
bgCount=bigramsCountList[0]
unigram_LM0 = ngram_LM(1, unigramsCountList[0], unigramVocabList[0])
bigram_LM0 = ngram_LM(2, bigramsCountList[0],bigramVocabList[0])
#create the Language model objects


In [52]:
unigram_LM0.test_Discount()
bigram_LM0.test_Discount()

TEST SUCCESSFUL!
TEST SUCCESSFUL!


In [56]:
print("The perplexity for ", testname[0], " on ", author[0], " using Unigram Discount with d 0.7 = "+ str(unigram_LM0.perplexity_discount(unigamsTestList[0],0.7)))
print("The perplexity for ", testname[1], " on ", author[0], " using Unigram Discount with d 0.7 = "+ str(unigram_LM0.perplexity_discount(unigamsTestList[1],0.7)))
print("The perplexity for ", testname[2], " on ", author[0], " using Unigram Discount with d 0.7 = "+ str(unigram_LM0.perplexity_discount(unigamsTestList[2],0.7)))
print("")
print("The perplexity for ", testname[0], " on ", author[0], " using Bigram Discount with d 0.7 = "+ str(bigram_LM0.perplexity_discount(bigramTestList[0],0.7)))
print("The perplexity for ", testname[1], " on ", author[0], " using Bigram Discount with d 0.7 = "+ str(bigram_LM0.perplexity_discount(bigramTestList[1],0.7)))
print("The perplexity for ", testname[2], " on ", author[0], " using Bigram Discount with d 0.7 = "+ str(bigram_LM0.perplexity_discount(bigramTestList[2],0.7)))

The perplexity for  seg1  on  dickens  using Unigram Discount with d 0.7 = 883.803412427612
The perplexity for  seg3  on  dickens  using Unigram Discount with d 0.7 = 772.8902558174975
The perplexity for  seg2  on  dickens  using Unigram Discount with d 0.7 = 831.429471634464

The perplexity for  seg1  on  dickens  using Bigram Discount with d 0.7 = 556.8732391518869
The perplexity for  seg3  on  dickens  using Bigram Discount with d 0.7 = 593.3775705136829
The perplexity for  seg2  on  dickens  using Bigram Discount with d 0.7 = 469.46272801977267


In [54]:
one_gram = unigamsList[1]
unigram_LM1 = ngram_LM(1, unigramsCountList[1], unigramVocabList[1])
bigram_LM1 = ngram_LM(2, bigramsCountList[1],bigramVocabList[1])
unigram_LM1.test_Discount()
bigram_LM1.test_Discount()

TEST SUCCESSFUL!
TEST SUCCESSFUL!


In [57]:
print("The perplexity for ", testname[0], " on ", author[1], " using Unigram Discount with d 0.7 = "+ str(unigram_LM1.perplexity_discount(unigamsTestList[0],0.7)))
print("The perplexity for ", testname[1], " on ", author[1], " using Unigram Discount with d 0.7 = "+ str(unigram_LM1.perplexity_discount(unigamsTestList[1],0.7)))
print("The perplexity for ", testname[2], " on ", author[1], " using Unigram Discount with d 0.7 = "+ str(unigram_LM1.perplexity_discount(unigamsTestList[2],0.7)))
print("")
print("The perplexity for ", testname[0], " on ", author[1], " using Bigram Discount with d 0.7 = "+ str(bigram_LM1.perplexity_discount(bigramTestList[0],0.7)))
print("The perplexity for ", testname[1], " on ", author[1], " using Bigram Discount with d 0.7 = "+ str(bigram_LM1.perplexity_discount(bigramTestList[1],0.7)))
print("The perplexity for ", testname[2], " on ", author[1], " using Bigram Discount with d 0.7 = "+ str(bigram_LM1.perplexity_discount(bigramTestList[2],0.7)))

The perplexity for  seg1  on  doyle  using Unigram Discount with d 0.7 = 909.425039609364
The perplexity for  seg3  on  doyle  using Unigram Discount with d 0.7 = 788.5189379725299
The perplexity for  seg2  on  doyle  using Unigram Discount with d 0.7 = 705.9961966525817

The perplexity for  seg1  on  doyle  using Bigram Discount with d 0.7 = 945.8472262120516
The perplexity for  seg3  on  doyle  using Bigram Discount with d 0.7 = 786.7886944331663
The perplexity for  seg2  on  doyle  using Bigram Discount with d 0.7 = 85.42554033468252


In [58]:
one_gram = unigamsList[2]
unigram_LM2 = ngram_LM(1, unigramsCountList[2], unigramVocabList[2])
bigram_LM2 = ngram_LM(2, bigramsCountList[2],bigramVocabList[2])
unigram_LM2.test_Discount()
bigram_LM2.test_Discount()

TEST SUCCESSFUL!
TEST SUCCESSFUL!


In [59]:
print("The perplexity for ", testname[0], " on ", author[2], " using Unigram Discount with d 0.7 = "+ str(unigram_LM2.perplexity_discount(unigamsTestList[0],0.7)))
print("The perplexity for ", testname[1], " on ", author[2], " using Unigram Discount with d 0.7 = "+ str(unigram_LM2.perplexity_discount(unigamsTestList[1],0.7)))
print("The perplexity for ", testname[2], " on ", author[2], " using Unigram Discount with d 0.7 = "+ str(unigram_LM2.perplexity_discount(unigamsTestList[2],0.7)))
print("")
print("The perplexity for ", testname[0], " on ", author[2], " using Bigram Discount with d 0.7 = "+ str(bigram_LM2.perplexity_discount(bigramTestList[0],0.7)))
print("The perplexity for ", testname[1], " on ", author[2], " using Bigram Discount with d 0.7 = "+ str(bigram_LM2.perplexity_discount(bigramTestList[1],0.7)))
print("The perplexity for ", testname[2], " on ", author[2], " using Bigram Discount with d 0.7 = "+ str(bigram_LM2.perplexity_discount(bigramTestList[2],0.7)))

The perplexity for  seg1  on  twain  using Unigram Discount with d 0.7 = 933.3029289033419
The perplexity for  seg3  on  twain  using Unigram Discount with d 0.7 = 551.0082009594672
The perplexity for  seg2  on  twain  using Unigram Discount with d 0.7 = 905.0855602930553

The perplexity for  seg1  on  twain  using Bigram Discount with d 0.7 = 921.756961225992
The perplexity for  seg3  on  twain  using Bigram Discount with d 0.7 = 74.60808802253757
The perplexity for  seg2  on  twain  using Bigram Discount with d 0.7 = 593.9379104755778


#Feature Selection(Extra Credit)

In [0]:
#merge the unigram and bigram into a universal list
import itertools
unigramTotal = list(itertools.chain.from_iterable(unigamsList))
bigramTotal = list(itertools.chain.from_iterable(bigramList))
#this is a unival count of unigrams and bigrams
unigramTotalCount = Counter(unigramTotal)
bigramTotalCount = Counter(bigramTotal)

In [0]:
#filtering the unigram and bigrams that occured less than 15times using a copied list of counts 
#Did not want to touch the existing count list
nw_unigramsCountList = unigramsCountList.copy()
nw_bigramsCountList = bigramsCountList.copy()
for i in range(0,3):
  nw_unigramsCountList[i] = Counter(dict(filter(lambda x: x[1] >= 15, nw_unigramsCountList[i].items())))
  nw_bigramsCountList[i] = Counter(dict(filter(lambda x: x[1] >= 15, nw_bigramsCountList[i].items())))
  
  unigPMI=nw_unigramsCountList.copy()
  bigPMI=nw_bigramsCountList.copy()
  for i in range(0,3):
    unigPMI[i]=dict(unigPMI[i])
    bigPMI[i]=dict(bigPMI[i])

In [0]:
def computePMI(f,c,type=1):
  #f is the feature
  #c is the category it is a number
  #type is the gram we are working with 1 for unigram, 2 for bigrams
  if type==1: #for unigram
    probabilityF=unigramsCountList[c][f]/len(unigamsList[c])
    probabilityC= 1/len(author)
    probabilityFC = unigramTotalCount[f]/len(unigramTotal)
    prob = probabilityFC/(probabilityF*probabilityC)
    return math.log(prob, 2)
  elif type==2: #for bigrams
    probabilityF=bigramsCountList[c][f]/len(bigramList[c])
    probabilityC= 1/len(author)
    probabilityFC = bigramTotalCount[f]/len(bigramTotal)
    prob = probabilityFC/(probabilityF*probabilityC)
    return  math.log(prob, 2)
    
  

In [0]:
for i in range(0,3):
  for word in unigPMI[i]:
      unigPMI[i][word] = computePMI(word,i,type=1)
  for word in bigPMI[i]:
      bigPMI[i][word] = computePMI(word,i,type=2)

In [0]:
#convert the PMI dict back to counter
for i in range(0,3):
    unigPMI[i]=Counter(unigPMI[i])
    bigPMI[i]=Counter(bigPMI[i])

In [0]:
#frequently occuring unigram
top10UnigPMI=[]
for unig in unigPMI:
  top10UnigPMI.append(unig.most_common(10))
  #frequently occuring unigram
top10BigPMI=[]
for bigrm in bigPMI:
  top10BigPMI.append(bigrm.most_common(10))

In [0]:
#this method prints the top 10 unigram with highest PMI
def printUnigramTop10PMI():
  for i in range(0, len(author)):
    print(author[i].upper())
    print("word\tPMI")
    for x  in top10UnigPMI[i]:
      print(x[0][0]+"\t"+str(x[1]))
    print("\n")

In [67]:
printUnigramTop10PMI()

DICKENS
word	PMI
professor	5.5636481157482995
warnt	4.977574139593979
de	4.860802490075416
maybe	4.4657780514347944
woods	4.2494764644953715
big	4.1944760296174
tom	4.160536571699934
dr	4.105188336178327
adventure	4.0727668584859495
huge	3.9979990900839772


DOYLE
word	PMI
jim	5.744230635642317
everybody	5.283178921336858
reckon	5.246237342948721
aint	5.173232282438711
walter	4.7800337185456545
king	4.733348054504782
nobody	4.577834086074116
somebody	4.533763840337158
de	4.503697115035013
tears	4.457635613867235


TWAIN
word	PMI
holmes	8.036661350642644
florence	6.191002710765577
watson	5.287765521240992
afterwards	4.9386444634630635
walter	4.933952768138341
james	4.529061394276328
mrs	4.382305206452185
visitor	4.28656776688337
marriage	4.270944242422864
returned	4.2443909739478




In [0]:
#this method prints the top 10 Bigram with highest PMI
def printBigramTop10PMI():
  for i in range(0, len(author)):
    print(author[i].upper())
    print("word\tPMI")
    for x  in top10BigPMI[i]:
      print(str(x[0])+"\t"+str(x[1]))
    print("\n")

In [69]:
printBigramTop10PMI()

DICKENS
word	PMI
('<s>', 'tom')	5.271385521312521
('the', 'thing')	4.235023597221317
('and', 'says')	4.081122220553853
('a', 'while')	4.067316421028823
('the', 'village')	4.020010706250466
('<s>', '</s>')	4.012032648826942
('cried', '</s>')	4.002732714818631
('he', 'cried')	3.971989191387523
('edge', 'of')	3.964869152058005
('we', 'could')	3.955994213379312


DOYLE
word	PMI
('and', 'said')	5.982270280330354
('the', 'captain')	5.8334137422021675
('said', 'mr')	5.0978915649056304
('the', 'king')	4.982367199818542
('said', 'mrs')	4.688726032258878
('the', 'boys')	4.3802547205804006
('and', 'went')	4.330373571903959
('went', 'on')	4.205188204141208
('on', 'and')	4.198658818196727
('the', 'people')	4.076291235655391


TWAIN
word	PMI
('my', 'dear')	5.9696060059148826
('<s>', 'cried')	5.211751166764461
('am', 'sure')	4.635961607838086
('<s>', 'asked')	4.629172442175447
('the', 'major')	4.5947389451115725
('mr', '</s>')	4.52428930262158
('appeared', 'to')	4.324105183556512
('my', 'friend')	4.3